This notebook can be ONLY used for:

1. Calculating the performance **upper bound: change pct_data = 1**.
2. Calculating the performance **lower bound: change pct_data = supervised data %**.

Make sure to add whatever file path you think will be more convinient. In my case, for each model I created a folder, and 3 subfolders, one for the LB, UP and Mean Teacher. If you choose this method, **make sure the folders are created beforehand**. Otherwise, feel free to do whatever. For this notebook, the following things will be saved: 

1. The weights of the model every 5 epochs (./whatever_name_you_want_epoch_X). 
2. Running loss during training (every epoch).
3. Accuracy and IOU on train set every 5 epochs.
4. Accuracy and IOU on validation set every 5 epochs.

**If you use this notebook directly from colab, PLEASE make a copy and don't change anything in the original file, cuz then my kernel will crash :(**.

**NOTE: This file uses a modified version of the dataloader in GitHub. If you run it with that one you will get an attribute error from get_data.**

In [1]:
# ONLY PARAMETER THAT SHOULD BE CHANGED. 
pct_data = 0.4 # Total amount of data the model will see. This is to calculate the LB (or UB) for the performance of the model. All labeled.
model_save_path = "./M2_29_03_23/M2_LB/"

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%ls

gdrive/  sample_data/


In [4]:
%cd /content/gdrive/MyDrive/Applied DL CW

/content/gdrive/MyDrive/Applied DL CW


In [5]:
%ls

data/                 M1_28_03_23/   __pycache__/              utils.py
data_augmentation.py  M2_29_03_23/   Train_Mean_Teacher.ipynb
data_into_loaders.py  model_UNet.py  Train_Supervised.ipynb


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
import model_UNet
from data_augmentation import augmentation, colorjiter, invert
import matplotlib.pyplot as plt
from data_into_loaders import get_data
from utils import dice_loss, wt, update_ema_variables, unsup_loss, evaluate_model

In [7]:
#### Do NOT touch these hyperparameters
img_resize = 64
depth, dropout_rate = 3, 0.25 # Depth of U-Net
val_pct, test_pct = 0.2, 0.1 # Validation and test set %. 

# Trainin params
batch_size = 32
epochs = 100
lr, lr_gamma = 1e-3, 0.9 #Optimizer params

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(f'Using device: {device}')

Using device: cuda:0


In [8]:
# Initialize models, losses and optimizers. Make sure to re-run this cell again if you stop training and start again
#Create 1 network
modelS = model_UNet.UNet(in_channels=3, num_classes=2, depth=depth)
modelS= modelS.to(device)

#optimizer
optimizer = Adam(modelS.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=lr_gamma, last_epoch=-1, verbose=True)

/content/gdrive/MyDrive/Applied DL CW/model_UNet.py:211: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
/content/gdrive/MyDrive/Applied DL CW/model_UNet.py:212: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


Adjusting learning rate of group 0 to 1.0000e-03.


In [11]:
mixed_train_loader, val_loader, test_loader = get_data(0.25,0.75,val_pct, test_pct, batch_size=batch_size, img_resize=img_resize, is_mixed_loader = False, pct_data = pct_data)

all images are =  7393
There are 7377 images.
There are 2065 train images, 1475 validation and 739 test


In [10]:
# Train
eval_freq = 5
losses, accsTr, IousTr, accsVal, IousVal = [], [], [], [], []

for epoch in range(epochs):

        modelS.train()
        running_loss = 0

        for step, data in enumerate(mixed_train_loader):

            imgs, labs = data
            # Augment images
            imgS_aug = augmentation(imgs)

            imgS_aug = imgS_aug.to(device)
            labs = labs.squeeze().type(torch.LongTensor).to(device)

            optimizer.zero_grad()

            # Forward pass for student and teacher
            z = modelS(imgS_aug) 
            loss = dice_loss(z, labs)
            
            loss.backward()
            
            optimizer.step()    
            running_loss += loss.item()


        print(f'Epoch: {epoch + 1:4d} - Loss: {running_loss:6.2f}')
        losses.append(running_loss)
    
        if (epoch % eval_freq == 0):

          # Get accuracy and IOU for train and validation dataset 
          accTr, IouTr = evaluate_model(modelS, mixed_train_loader, device)
          accVal, IouVal = evaluate_model(modelS, val_loader, device)
          
          accsTr.append(accTr)
          IousTr.append(IouTr)   
          accsVal.append(accVal)
          IousVal.append(IouVal)

          print(f'For training: accuracy-{accTr:2.0%}; IOU-{IouTr:2.0%}')
          print(f'For validation: accuracy-{accVal:2.0%}; IOU-{IouVal:2.0%}')

          np.savetxt(f"{model_save_path}running_loss", losses)
          np.savetxt(f"{model_save_path}train_accuracy", accsTr)
          np.savetxt(f"{model_save_path}train_IOU", IousTr)
          np.savetxt(f"{model_save_path}val_accuracy", accsVal)
          np.savetxt(f"{model_save_path}val_IOU", IousVal)

          torch.save(modelS.state_dict(), f"{model_save_path}model_epoch_{epoch+1}" + '.pt')


Epoch:    1 - Loss:  17.81


KeyboardInterrupt: ignored